In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import plotly.express as px
from datetime import datetime as dt, timedelta
import pylab 
import scipy.stats as st
import statsmodels.stats.api as sms
import random
from tqdm.notebook import tqdm
from scipy.stats import norm, mannwhitneyu
from scipy.stats import ttest_ind
from tqdm.auto import tqdm
from scipy.stats import levene
import warnings
import statsmodels.api as sm
from scipy.stats._continuous_distns import _distn_names
import matplotlib
from scipy.stats import recipinvgauss
from pandasql import sqldf
import requests
from urllib.parse import urlencode
import sqlite3

C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Задание 2. SQL

2.1.1 Условие

Образовательные курсы состоят из различных уроков,
каждый из которых состоит из нескольких маленьких заданий.
Каждое такое маленькое задание называется "горошиной".
Назовём очень усердным учеником того пользователя, 
который хотя бы раз за текущий месяц правильно решил 20 горошин за час.

Необходимо написать оптимальный запрос, 
который даст информацию о количестве очень усердных студентов за март 2020 года.
NB! Под усердным студентом мы понимаем студента, 
который правильно решил 20 задач за текущий месяц.

SQL-выражение
Синтаксической разбор
Компоновка
Оптимизация запроса
Выполнение запроса
Результаты запроса

1. FROM
2. ON
3. JOIN
4. WHERE
5. GROUP BY
6. WITH CUBE или WITH ROLLUP
7. HAVING
8. SELECT
10. ORDER BY
11. TOP
12. SELECT (возврат результирующего датасета).


2. Понимание как храняться данные и выбираются(индексы, порциции)
3. Знания как физически выполняется JOIN(nested loops, hash and merge join)
4. Базовые принцыпы оптимизации.

In [ ]:
#con = sqlite3.connect('/content/drive/MyDrive/my_data/progect_1')

In [ ]:
#df.to_sql('progect_1',con,index=False,if_exists='replace')

In [2]:
# Генерируем таблицу peas.
n = 1333
peas = pd.DataFrame({'st_id': np.random.poisson(400,size=n)})

start_date = pd.to_datetime('2020-03-01')
times_start = pd.date_range(start_date, periods=n, freq='30min')

peas['times_start'] = times_start

peas['timest'] = np.random.poisson(2,size=n)

peas['correct'] = pd.Series([True, True, True, True, True, True, True, True, True, False, False]) \
    .sample(len(peas),replace=True,random_state=42) \
    .values

peas['subject'] = pd.Series(['math', 'economics', 'sociology', 'philosophy', 'physics']) \
    .sample(len(peas),replace=True,random_state=42) \
    .values
peas.head()

,st_id,times_start,timest,correct,subject
0,380,2020-03-01 00:00:00,1,True,philosophy
1,402,2020-03-01 00:30:00,0,True,physics
2,376,2020-03-01 01:00:00,2,False,sociology
3,419,2020-03-01 01:30:00,1,True,physics
4,381,2020-03-01 02:00:00,1,True,physics


In [3]:
# 1. Когорты по месяцам.
# 2. Ученик правильно решил.
# 3. Решил 20 горошин.
# 4. Какое количество таких учеников в когорте за март 2020.
# решение №1
q = """--EXPLAIN QUERY PLAN 
          WITH  peas_activ_march as (SELECT strftime('%Y-%m', times_start) as month,
                  count(st_id)         
          FROM peas
          WHERE correct = 1 and month ='2020-03'
          GROUP BY month, st_id
          HAVING count(st_id) >= 20)
      SELECT COUNT(*) AS active_students_March
      FROM peas_activ_march
     
        """

In [4]:
sqldf(q)

,active_students_March
0,18


In [5]:
# решение №2
q = """--EXPLAIN QUERY PLAN
       SELECT  COUNT(t.st_id) as active_students_March -- выводит результат количества учеников больше 20
       FROM (SELECT strftime('%Y-%m', times_start) as month, -- приводит время начала выполнения к году и мецяцу
                    SUM(correct) as correct, -- считает сумму правильных ответов
                    st_id -- выводит ид учеников
            FROM peas
            GROUP BY st_id, month, correct
            HAVING month = '2020-03'
            ) t
        WHERE t.correct >= 20 -- условие что правильных решений больше 20
        """

In [6]:
sqldf(q)

,active_students_March
0,18


2.2 Оптимизация воронки
2.2.1 Условие
Образовательная платформа предлагает пройти студентам курсы по модели trial:
студент может решить бесплатно лишь 30 горошин в день.
Для неограниченого количества заданий в определенной дисциплине
студенту необходимо приобрести полный доступ. Команда провела эксперимент,
где был протестирован новый экран оплаты.

In [7]:
# Генерируем таблицу studs.
n = 700
studs = pd.DataFrame({'st_id': np.arange(n)})

studs['test_grp'] = pd.Series(['A', 'A', 'A', 'B', 'B']) \
    .sample(len(studs),replace=True,random_state=42) \
    .values
studs.head()

,st_id,test_grp
0,0,B
1,1,B
2,2,A
3,3,B
4,4,B


In [8]:
# Генерируем таблицу checks.
checks = pd.DataFrame({'st_id': peas['st_id'].sample(145).unique()})

start_date = pd.to_datetime('2020-03-01')
checks['sale_time'] = pd.date_range(start_date, periods=len(checks), freq='12H')

checks['subject'] = pd.Series(['math', 'economics', 'sociology', 'philosophy', 'physics']) \
    .sample(len(checks),replace=True,random_state=42) \
    .values

checks_price = pd.DataFrame({'math':[1000], 'economics':[750], 'sociology':[500], 'philosophy':[450], 'physics':[600]}) \
    .T \
    .reset_index() \

checks_price.columns = ['subject','subject_price']

checks = checks.merge(checks_price,how='left',on='subject')
checks.head()

,st_id,sale_time,subject,subject_price
0,405,2020-03-01 00:00:00,philosophy,450
1,363,2020-03-01 12:00:00,physics,600
2,376,2020-03-02 00:00:00,sociology,500
3,389,2020-03-02 12:00:00,physics,600
4,396,2020-03-03 00:00:00,physics,600


Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
ARPU=Конверсия*ARPPU
    Конверсия = Число пользователей / Число клиентов
    ARPPU = Доход / Число клиентов 
ARPAU - Сумма общего дохода за месяц / Общая ежемесячная база активных пользователей.
CR количество пользователей, которые выполнили какое-либо целевое действие / Общее число пользователей.
СR активного пользователя в покупку 
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

In [9]:
peas.head()

,st_id,times_start,timest,correct,subject
0,380,2020-03-01 00:00:00,1,True,philosophy
1,402,2020-03-01 00:30:00,0,True,physics
2,376,2020-03-01 01:00:00,2,False,sociology
3,419,2020-03-01 01:30:00,1,True,physics
4,381,2020-03-01 02:00:00,1,True,physics


In [10]:
checks.head()

,st_id,sale_time,subject,subject_price
0,405,2020-03-01 00:00:00,philosophy,450
1,363,2020-03-01 12:00:00,physics,600
2,376,2020-03-02 00:00:00,sociology,500
3,389,2020-03-02 12:00:00,physics,600
4,396,2020-03-03 00:00:00,physics,600


In [11]:
studs.head()

,st_id,test_grp
0,0,B
1,1,B
2,2,A
3,3,B
4,4,B


In [12]:
q = """ WITH 
            -- Создаем временную таблицу с началом и окончанием времени эксперимента.
            checks_exp as (SELECT min(sale_time) as checks_start,
                                  max(sale_time) as checks_finish
                           FROM checks)
            ,
            -- Создаем временную таблицу peas в рамках проведения нашего эксперимента.            
            peas_unique as (SELECT st_id,
                                   1 as active, -- Добавляем столбец с активностью.
                                   subject as subject_active
                            FROM peas
                            WHERE times_start > (SELECT checks_start
                                                 FROM checks_exp)
                                  AND 
                                  times_start < (SELECT checks_finish
                                                 FROM checks_exp)
                            GROUP BY st_id)
            ,
            -- Джойним таблицы studs и peas_unique.
            studs_group_activ as (SELECT *
                                  FROM studs s
                                  LEFT JOIN peas_unique p ON s.st_id = p.st_id)
            ,
            -- Джойним таблицы studs_group_activ и checks.
            data_for_exp as (SELECT *
                             FROM studs_group_activ s
                             LEFT JOIN checks c ON s.st_id = c.st_id)
            ,
            -- Создаем таблицу с пользователями, которые были активны по математике.
            math_activ as (SELECT st_id,
                                  1 as math_activ
                           FROM data_for_exp
                           WHERE subject_active = "math")
            ,
            -- Создаем таблицу с пользователями, которые были активны по математике, и купили полный доступ.
            math_activ_pay as (SELECT st_id,
                                  1 as math_activ_pay
                           FROM data_for_exp
                           WHERE subject = "math"
                                 AND
                                 subject_active = "math")
            ,
            -- Джойним данные с активностью по математике к основной таблице.
            data_for_exp1 as (SELECT *
                             FROM data_for_exp s
                             LEFT JOIN math_activ c ON s.st_id = c.st_id)
            ,
            -- Джойним данные с активностью по математике и покупке полного доступа по математике.
            data_for_exp2 as (SELECT *
                             FROM data_for_exp1 s
                             LEFT JOIN math_activ_pay c ON s.st_id = c.st_id)
            ,
            -- Собираем только нужные столбцы для подсчета метрик в одну таблицу.
            result as (SELECT st_id,
                              test_grp,
                              active,
                              sale_time,
                              subject,
                              subject_price,
                              subject_active,
                              1 as user,
                              math_activ_pay,
                              math_activ
                       FROM data_for_exp2)
                       
      -- Подсчитываем нужные нам метрики.                     
      SELECT test_grp,
             SUM(subject_price) / SUM(active) as ARPAU,
             CAST(COUNT(st_id) / COUNT(subject_price) AS INT) *
             SUM(subject_price) / COUNT(subject_price) AS ARPU,
             CAST((CAST(COUNT(subject_price) AS Float64) / COUNT(st_id)) AS DECIMAL(5,2)) AS CR,
             CAST((CAST(COUNT(subject_price) AS Float64) / SUM(active)) AS DECIMAL(5,2)) AS CR_activ,
             CAST((CAST(SUM(math_activ_pay) AS Float64) /
             SUM(math_activ)) AS DECIMAL(5,2)) AS CR_activ_math_in_math_pay
             -- Данные для проверки вычислений.
             --SUM(user) as user_sum,
             --SUM(math_activ_pay) as math_activ_pay,
             --SUM(math_activ) as math_activ
             --COUNT(st_id) AS st_id_count ,
             --SUM(active) AS active_user_sum ,
             --COUNT(subject_price) AS subject_price_count
      FROM result
      GROUP BY test_grp
     """

In [13]:
sqldf(q)

,test_grp,ARPAU,ARPU,CR,CR_activ,CR_activ_math_in_math_pay
0,A,440,7134,0.089109,0.679245,NaN
1,B,458,4252,0.135135,0.754717,0.090909
